# 操作系统

## 1 select,poll和epoll

其实所有的I/O都是轮询的方法,只不过实现的层面不同罢了.

这个问题可能有点深入了,但相信能回答出这个问题是对I/O多路复用有很好的了解了。其中tornado使用的就是epoll的.

[selec,poll和epoll区别总结](http://www.cnblogs.com/Anker/p/3265058.html)

基本上select有3个缺点:

1. 连接数受限
2. 查找配对速度慢
3. 数据由内核拷贝到用户态

poll改善了第一个缺点

epoll改了三个缺点.

关于epoll的: http://www.cnblogs.com/my_life/articles/3968782.html

> select、poll 和 epoll 是操作系统提供的 I/O 多路复用机制，用于同时监控多个文件描述符的状态变化，避免阻塞式 I/O 带来的性能问题。

1. select
- 工作原理
    - 使用位图（bitmap）来标识文件描述符集合
    - 内核遍历所有监控的文件描述符，检查是否有 I/O 事件发生
    - 如果没有事件发生，进程进入睡眠状态
- 优点：
    - 跨平台性好，几乎所有操作系统都支持
    - 使用简单
- 缺点：
    - 文件描述符数量有限制（通常为 1024）
    - 每次调用都需要将 fd_set 从用户空间拷贝到内核空间
    - 内核需要线性扫描所有监控的文件描述符
    - 返回后需要遍历整个 fd_set 找到就绪的文件描述符

2. poll
- 工作原理
    - 使用结构体数组代替位图
    - 内核遍历 pollfd 数组，检查每个文件描述符的状态
    - 没有最大文件描述符数量的硬性限制
- 优点：
    - 没有最大文件描述符数量限制
    - 使用结构体，更加清晰和灵活
- 缺点：
    - 仍需要将 pollfd 数组从用户空间拷贝到内核空间
    - 内核仍需要线性扫描所有文件描述符
    - 返回后仍需要遍历数组找到就绪的文件描述符

3. epoll
- 工作原理
    - 在内核中维护一个事件表
    - 使用红黑树存储监控的文件描述符
    - 使用链表存储就绪的文件描述符
    - 支持边缘触发（ET）和水平触发（LT）两种模式
- 优点：
    - 没有文件描述符数量限制
    - I/O 效率高，时间复杂度 O(1)
    - 只返回就绪的文件描述符，无需遍历
    - 支持边缘触发，减少系统调用次数
    - 内核与用户空间共享内存，减少拷贝开销
- 缺点：
    - 只在 Linux 系统上可用
    - 编程复杂度相对较高

## 2 调度算法

1. 先来先服务(FCFS, First Come First Serve)
2. 短作业优先(SJF, Shortest Job First)
3. 最高优先权调度(Priority Scheduling)
4. 时间片轮转(RR, Round Robin)
5. 多级反馈队列调度(multilevel feedback queue scheduling)

常见的调度算法总结:http://www.jianshu.com/p/6edf8174c1eb

实时调度算法:

1. 最早截至时间优先 EDF
2. 最低松弛度优先 LLF

## 3 死锁
原因:

1. 竞争资源
2. 程序推进顺序不当

必要条件:

1. 互斥条件
2. 请求和保持条件
3. 不剥夺条件
4. 环路等待条件

处理死锁基本方法:

1. 预防死锁(摒弃除1以外的条件)
2. 避免死锁(银行家算法)
3. 检测死锁(资源分配图)
4. 解除死锁
    1. 剥夺资源
    2. 撤销进程

死锁概念处理策略详细介绍:https://wizardforcel.gitbooks.io/wangdaokaoyan-os/content/10.html

## 4 程序编译与链接

推荐: http://www.ruanyifeng.com/blog/2014/11/compiler.html

Bulid过程可以分解为4个步骤:预处理(Prepressing), 编译(Compilation)、汇编(Assembly)、链接(Linking)

以c语言为例:

### 1 预处理

预编译过程主要处理那些源文件中的以“#”开始的预编译指令，主要处理规则有：

1. 将所有的“#define”删除，并展开所用的宏定义
2. 处理所有条件预编译指令，比如“#if”、“#ifdef”、 “#elif”、“#endif”
3. 处理“#include”预编译指令，将被包含的文件插入到该编译指令的位置，注：此过程是递归进行的
4. 删除所有注释
5. 添加行号和文件名标识，以便于编译时编译器产生调试用的行号信息以及用于编译时产生编译错误或警告时可显示行号
6. 保留所有的#pragma编译器指令。

### 2 编译

编译过程就是把预处理完的文件进行一系列的词法分析、语法分析、语义分析及优化后生成相应的汇编代码文件。这个过程是整个程序构建的核心部分。

### 3 汇编

汇编器是将汇编代码转化成机器可以执行的指令，每一条汇编语句几乎都是一条机器指令。经过编译、链接、汇编输出的文件成为目标文件(Object File)

### 4 链接

链接的主要内容就是把各个模块之间相互引用的部分处理好，使各个模块可以正确的拼接。
链接的主要过程包块 地址和空间的分配（Address and Storage Allocation）、符号决议(S

## 5 静态链接和动态链接

静态链接方法：静态链接的时候，载入代码就会把程序会用到的动态代码或动态代码的地址确定下来
静态库的链接可以使用静态链接，动态链接库也可以使用这种方法链接导入库

动态链接方法：使用这种方式的程序并不在一开始就完成动态链接，而是直到真正调用动态库代码时，载入程序才计算(被调用的那部分)动态代码的逻辑地址，然后等到某个时候，程序又需要调用另外某块动态代码时，载入程序又去计算这部分代码的逻辑地址，所以，这种方式使程序初始化时间较短，但运行期间的性能比不上静态链接的程序


## 6 虚拟内存技术

虚拟存储器是指具有请求调入功能和置换功能,能从逻辑上对内存容量加以扩充的一种存储系统.

## 7 分页和分段

分页: 用户程序的地址空间被划分成若干固定大小的区域，称为“页”，相应地，内存空间分成若干个物理块，页和块的大小相等。可将用户程序的任一页放在内存的任一块中，实现了离散分配。

分段: 将用户程序地址空间分成若干个大小不等的段，每段可以定义一组相对完整的逻辑信息。存储分配时，以段为单位，段与段在内存中可以不相邻接，也实现了离散分配。

### 分页与分段的主要区别

1. 页是信息的物理单位,分页是为了实现非连续分配,以便解决内存碎片问题,或者说分页是由于系统管理的需要.段是信息的逻辑单位,它含有一组意义相对完整的信息,分段的目的是为了更好地实现共享,满足用户的需要.
2. 页的大小固定,由系统确定,将逻辑地址划分为页号和页内地址是由机器硬件实现的.而段的长度却不固定,决定于用户所编写的程序,通常由编译程序在对源程序进行编译时根据信息的性质来划分.
3. 分页的作业地址空间是一维的、分段的地址空间是二维的。


## 8 页面置换算法

页面置换：在地址映射过程中，若所要访问的页面不在内存中，则产生了‘缺页中断(page fault)’。此时操作系统必须在内存中选择一个页面将其移出内存，为即将调入的页面让出空间。
1. 最佳置换算法 OPT (optional replacement)：被替换的页面为在未来最长时间内不会被访问的页面，可保证最低的缺页率，但不可能实现，主要用于评估算法。
2. 先进先出 FIFO：最易实现，但会频繁换页，性能差。
3. 最近最久未使用算法 LRU (Least Recently Used)：最近一段时间里最久没有使用过的页面予以置换。
4. 时钟替换算法 (Clock)：依照使用位替换页面。

## 9 边沿触发和水平触发

1. 边沿触发 (Edge Trigger)：自上次状态改变后有新的 I/O 事件就会触发通知，需要尽可能多的执行 I/O 操作。
2. 水平触发 (Level Trigger)：准备就绪时（可非阻塞地执行 I/O 系统调用）触发通知，可在任意时刻重复检测 I/O 状态。